In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv


In [16]:
# change these 3 to generate flights to each city for the date from the specific airport
departloc = "MSN"
depart_time = "2025-01-04"
return_time = "2025-01-06"

def generate_flight_links(departloc, depart_time, return_time, team_airports):
    flight_links = {}
    
    for team, arriveloc in team_airports.items():
        flight_url = f"https://flights.booking.com/flights/{departloc}.AIRPORT-{arriveloc}.AIRPORT/?type=ROUNDTRIP&adults=1&cabinClass=ECONOMY&children=&from=MSN.AIRPORT&to={arriveloc}.AIRPORT&fromCountry=US&toCountry=US&fromLocationName=Dane+County+Regional+Airport&toLocationName=Green+Bay%E2%80%93Austin+Straubel+International+Airport&depart={depart_time}&return={return_time}&sort=BEST&travelPurpose=leisure&aid=304142&label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4AvnvprsGwAIB0gIkMGI3ZTY2OTItMjdlYS00M2JjLThiZTAtOTRjODRhMDA5NzVh2AIE4AIB&adplat=www-searchresults_irene-web_shell_header-flight-missing_creative-5VHWvGC1SMhs4k08O29emO&stops=1"
        flight_links[team] = flight_url

    return flight_links

team_airports = {
    "MIN": "MSP",  # Minnesota Vikings - Minneapolis-St. Paul International
    "CHI": "ORD",  # Chicago Bears - O'Hare International
    "GB": "GRB",   # Green Bay Packers - Green Bay–Austin Straubel International
    "DET": "DTW",  # Detroit Lions - Detroit Metropolitan
    "IND": "IND",  # Indianapolis Colts - Indianapolis International
    "JAX": "JAX",  # Jacksonville Jaguars - Jacksonville International
    "TEN": "BNA",  # Tennessee Titans - Nashville International
    "NE": "BOS",   # New England Patriots - Logan International
    "BUF": "BUF",  # Buffalo Bills - Buffalo Niagara International
    "MIA": "MIA",  # Miami Dolphins - Miami International
    "NYJ": "JFK",  # New York Jets - John F. Kennedy International
    "BAL": "BWI",  # Baltimore Ravens - Baltimore/Washington International
    "CIN": "CVG",  # Cincinnati Bengals - Cincinnati/NKY International
    "CLE": "CLE",  # Cleveland Browns - Cleveland Hopkins International
    "PIT": "PIT",  # Pittsburgh Steelers - Pittsburgh International
    "HOU": "IAH",  # Houston Texans - George Bush Intercontinental
    "KC": "MCI",   # Kansas City Chiefs - Kansas City International
    "LV": "LAS",   # Las Vegas Raiders - McCarran International
    "LAC": "LAX",  # Los Angeles Chargers - Los Angeles International
    "LAR": "LAX",  # Los Angeles Rams - Los Angeles International
    "SEA": "SEA",  # Seattle Seahawks - Seattle-Tacoma International
    "ARI": "PHX",  # Arizona Cardinals - Phoenix Sky Harbor International
    "SF": "SFO",   # San Francisco 49ers - San Francisco International
    "ATL": "ATL",  # Atlanta Falcons - Hartsfield-Jackson Atlanta International
    "CAR": "CLT",  # Carolina Panthers - Charlotte Douglas International
    "NO": "MSY",   # New Orleans Saints - Louis Armstrong New Orleans International
    "TB": "TPA",   # Tampa Bay Buccaneers - Tampa International
    "NYG": "JFK",  # New York Giants - John F. Kennedy International
    "DAL": "DFW",  # Dallas Cowboys - Dallas/Fort Worth International
    "WAS": "DCA",  # Washington Commanders - Ronald Reagan Washington National
    "PHI": "PHL",  # Philadelphia Eagles - Philadelphia International
    "CHI": "ORD",  # Chicago Bears - O'Hare International
    "MIN": "MSP",  # Minnesota Vikings - Minneapolis-St. Paul International
}

# flight_links['MIN'] would ouput the link FROM the chosen departloc TO min
flight_links = generate_flight_links(departloc, depart_time, return_time, team_airports)


In [18]:
flight_links['CAR']
nfc_north_codes = ['MIN', 'CHI', 'DET', 'GB']
nfc_east_codes = ['PHI', 'WSH', 'DAL', 'NYG']
nfc_south_codes = ['TB', 'ATL', 'NO', 'CAR']
nfc_west_codes = ['LAR', 'SEA', 'ARI', 'SF']
afc_west_codes = ['KC', 'LAC', 'DEN', 'LV']
afc_east_codes = ['BUF', 'MIA', 'NYJ', 'NE']
afc_south_codes = ['HOU', 'IND', 'JAX', 'TEN']
afc_north_codes = ['PIT', 'BAL', 'CIN', 'CLE']
all_codes = nfc_north_codes + nfc_east_codes + nfc_south_codes + nfc_west_codes + afc_west_codes + afc_east_codes + afc_south_codes + afc_north_codes

https://flights.booking.com/flights/MSN.AIRPORT-CLT.AIRPORT/?type=ROUNDTRIP&adults=1&cabinClass=ECONOMY&children=&from=MSN.AIRPORT&to=CLT.AIRPORT&fromCountry=US&toCountry=US&fromLocationName=Dane+County+Regional+Airport&toLocationName=Green+Bay%E2%80%93Austin+Straubel+International+Airport&depart=2025-01-04&return=2025-01-06&sort=BEST&travelPurpose=leisure&aid=304142&label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4AvnvprsGwAIB0gIkMGI3ZTY2OTItMjdlYS00M2JjLThiZTAtOTRjODRhMDA5NzVh2AIE4AIB&adplat=www-searchresults_irene-web_shell_header-flight-missing_creative-5VHWvGC1SMhs4k08O29emO&stops=1


In [67]:
def scrape_flight_details(url, team_name):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Send the request and check for success
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return []
    
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # List to store flight details
    flight_details = []

    # Extract flight information containers
    flights = soup.select(".Tab-module__item-panel___Z9erA.Tab-module__item-panel--active___5rRzY")

    for flight in flights:
        try:
            # Extract airline name
            airline_element = flight.select_one(".Frame-module__margin-top_2___9V-am")  # Update with the correct class
            airline_name = airline_element.get_text(strip=True) if airline_element else "N/A"
            print(airline_name)

            # Extract departure and arrival times for the outgoing flight (to destination)
            departure_time_outbound_element = flight.select_one(".Text-module__root--variant-strong_1___L2d5B")  # Update with correct class
            arrival_time_outbound_element = flight.select_one(".Text-module__root--variant-strong_1___L2d5B")  # Update with correct class
            departure_time_outbound = departure_time_outbound_element.get_text(strip=True) if departure_time_outbound_element else "N/A"
            arrival_time_outbound = arrival_time_outbound_element.get_text(strip=True) if arrival_time_outbound_element else "N/A"

            # Extract departure and arrival times for the return flight
            departure_time_return_element = flight.select_one(".departure-time-return-selector")  # Update with correct class
            arrival_time_return_element = flight.select_one(".arrival-time-return-selector")  # Update with correct class
            departure_time_return = departure_time_return_element.get_text(strip=True) if departure_time_return_element else "N/A"
            arrival_time_return = arrival_time_return_element.get_text(strip=True) if arrival_time_return_element else "N/A"

            # Extract price
            price_element = flight.select_one(".price-selector")  # Update with the correct class
            price_text = price_element.get_text(strip=True) if price_element else "N/A"
            flight_price = int("".join(filter(str.isdigit, price_text))) if price_text != "N/A" else "N/A"

            # Append the details to the list
            flight_details.append({
                "Team": team_name,
                "Airline": airline_name,
                "Departure Time (Outbound)": departure_time_outbound,
                "Arrival Time (Outbound)": arrival_time_outbound,
                "Departure Time (Return)": departure_time_return,
                "Arrival Time (Return)": arrival_time_return,
                "Price": flight_price
            })
        except Exception as e:
            print(f"Error processing flight: {e}")
            continue

    return flight_details


In [68]:
url = "https://flights.booking.com/flights/MSN.AIRPORT-GRB.AIRPORT/?type=ROUNDTRIP&adults=1&cabinClass=ECONOMY&children=&from=MSN.AIRPORT&to=GRB.AIRPORT&fromCountry=US&toCountry=US&fromLocationName=Dane+County+Regional+Airport&toLocationName=Green+Bay%E2%80%93Austin+Straubel+International+Airport&depart=2025-01-04&return=2025-01-06&sort=BEST&travelPurpose=leisure&aid=304142&label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4AvnvprsGwAIB0gIkMGI3ZTY2OTItMjdlYS00M2JjLThiZTAtOTRjODRhMDA5NzVh2AIE4AIB&adplat=www-searchresults_irene-web_shell_header-flight-missing_creative-5VHWvGC1SMhs4k08O29emO&stops=1"
team_name = "GB"
flights = scrape_flight_details(url, team_name)

# Output the details
for flight in flights:
    print(flight)


N/A
{'Team': 'GB', 'Airline': 'N/A', 'Departure Time (Outbound)': 'N/A', 'Arrival Time (Outbound)': 'N/A', 'Departure Time (Return)': 'N/A', 'Arrival Time (Return)': 'N/A', 'Price': 'N/A'}
